# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities_df = pd.read_csv("city_weather.csv")

cities_df.head()

,city,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,punta arenas,75.0,CL,1.635129e+09,60.0,-53.1500,-70.9167,4.06,10.80
1,humberto de campos,19.0,BR,1.635129e+09,86.0,-2.5983,-43.4611,25.67,2.76
2,saint-philippe,48.0,RE,1.635128e+09,81.0,-21.3585,55.7679,22.75,6.33
3,mataura,90.0,NZ,1.635129e+09,52.0,-46.1927,168.8643,22.09,2.19
4,paita,58.0,PE,1.635129e+09,82.0,-5.0892,-81.1144,18.76,5.83


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [15]:
#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

formatted_cities = cities_df.dropna()

weather_fitted = formatted_cities.loc[(formatted_cities['Cloudiness'] > 0) & (formatted_cities['Wind Speed'] < 10)] 
weather_fitted_final = weather_fitted.loc[(weather_fitted['Max Temp'] > 70) & (weather_fitted['Max Temp'] <= 80)]
weather_fitted_final

,city,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df =  pd.DataFrame(weather_fitted_final, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [17]:
# Print the name and address of the first restaurant that appears
hotel_name = []

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [18]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()

,City,Country,Lat,Lng,Hotel Name


In [19]:
hotel_df.to_csv('hotels.csv')

In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]

SyntaxError: unexpected EOF while parsing (<ipython-input-20-669900fb8366>, line 14)

In [21]:
# Display figure
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

InvalidPointException: nan is not a valid latitude. Latitudes must lie between -90 and 90.